In [1]:
import sys
import getpass
import re
from functools import partial
from tqdm import tqdm
import uuid

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *  # initial imports
reload_module('utils_io')  # reload after update to utils_io.py
from utils_io import *  # reload reflects updates

#geo imports
import geopandas as gp
import folium
from geopandas.tools import geocode
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from geopy import Point

In [2]:
google_api_key = G_CREDS['dataviz_team']

## Pull Geocoded Data and Run Through Geocoder w/ updates

In [3]:
#Create a geo locator
g = GoogleV3(api_key=google_api_key)

In [ ]:
components = (components={"country": "USA"},
                                bounds=bound_box,
                               timeout=1000)

In [17]:
#Use a rate limiter
bound_box = [Point(38.864245,-121.208156), Point(36.893329,-123.632497)]
geocode = RateLimiter(g.geocode, 
                      min_delay_seconds=.025, 
                      error_wait_seconds=1)

In [18]:
geocode

In [57]:
address_list = ['757 SHULLSBURG WAY, OAKLEY, CA',
                '749 SHULLSBURG WAY, OAKLEY, CA',
                '745 SHULLSBURG WAY, OAKLEY, CA',
                '133 SISTERS CT, LOS GATOS, CA',
                '3 QUETZAL CT, REDWOOD CITY-0000, SAN MATEO COUNTY, CA 94062-0000',
                '133 SISTERS CT, LOS GATOS, CA, USA']
juris_list = ['OAKLEY','OAKLEY','OAKLEY','LOS GATOS','SAN MATEO COUNTY','LOS GATOS']

In [58]:
#Create an example dataset
data = {'address':address_list,'jurisdiction':juris_list}

In [59]:
ex_addresses = pd.DataFrame(data)

In [60]:
ex_addresses.head(5)

,address,jurisdiction
0,"757 SHULLSBURG WAY, OAKLEY, CA",OAKLEY
1,"749 SHULLSBURG WAY, OAKLEY, CA",OAKLEY
2,"745 SHULLSBURG WAY, OAKLEY, CA",OAKLEY
3,"133 SISTERS CT, LOS GATOS, CA",LOS GATOS
4,"3 QUETZAL CT, REDWOOD CITY-0000, SAN MATEO COU...",SAN MATEO COUNTY


In [91]:
#Create a location column with geolocator and time the process 
tqdm.pandas()
bound_box = [Point(38.864245,-121.208156), Point(36.893329,-123.632497)]
ex_addresses['location'] = (ex_addresses['address']
                            .progress_apply(partial(geocode,
                                                    components={"country": "USA"},
                                                    timeout=1000,
                                                    bounds=bound_box)))






100%|██████████| 6/6 [00:04<00:00,  1.47it/s]


In [97]:
ex_addresses['location'].iloc[0].raw

{'address_components': [{'long_name': 'Shullsburg',
   'short_name': 'Shullsburg',
   'types': ['locality', 'political']},
  {'long_name': 'Lafayette County',
   'short_name': 'Lafayette County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Wisconsin',
   'short_name': 'WI',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '53586', 'short_name': '53586', 'types': ['postal_code']}],
 'formatted_address': 'Shullsburg, WI 53586, USA',
 'geometry': {'bounds': {'northeast': {'lat': 42.580734, 'lng': -90.2211169},
   'southwest': {'lat': 42.566026, 'lng': -90.2463861}},
  'location': {'lat': 42.5733354, 'lng': -90.23096},
  'location_type': 'APPROXIMATE',
  'viewport': {'northeast': {'lat': 42.580734, 'lng': -90.2211169},
   'southwest': {'lat': 42.566026, 'lng': -90.2463861}}},
 'partial_match': True,
 'place_id': 'ChIJX-tzKGZL_YcR4yeseAjHSN

In [95]:
ex_addresses.iloc[0]

address                            757 SHULLSBURG WAY, OAKLEY, CA
jurisdiction                                               OAKLEY
location        (Shullsburg, WI 53586, USA, (42.5733354, -90.2...
Name: 0, dtype: object